# [Issue 4850](https://github.com/OpenMined/PySyft/issues/4850)

## [Hello world notebook for zksk](https://github.com/spring-epfl/zksk)

In [1]:
from zksk import Secret, DLRep
from zksk import utils

# Setup: DO and DS agree on two group generators.
G, H = utils.make_generators(num=2, seed=42)

print(type(G))
G = utils.make_generators(num=1, seed=42)
print(type(G))
# Setup: generate a secret randomizer.
r = Secret(utils.get_random_num(bits=128))

# This is DO's secret bit.
top_secret_bit = 1

# A Pedersen commitment to the secret bit.
C = top_secret_bit * G + r.value * H

# DO's definition of the proof statement, and proof generation.
# (The first or-clause corresponds to the secret value 0, and the second to the value 1. Because
# the real value of the bit is 1, the clause that corresponds to zero is marked as simulated.)
stmt = DLRep(C, r * H, simulated=True) | DLRep(C - G, r * H)
zk_proof = stmt.prove()

<class 'petlib.ec.EcPt'>
<class 'list'>


TypeError: can only concatenate list (not "EcPt") to list

In [2]:
# DS will receive the commitment C and zk_proof from DO, and run this to verify the proof:
from zksk import Secret, DLRep

# Setup: get the agreed group generators.
G, H = utils.make_generators(num=2, seed=42)
# Setup: define a randomizer with an unknown value.
print(dir(G))
r = Secret()

stmt = DLRep(C, r * H) 
stmt = DLRep(C, r * H) | DLRep(C - G, r * H)
assert stmt.verify(zk_proof)

['__add__', '__add_inplace__', '__class__', '__copy__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__neg__', '__neg_inplace__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rmul_inplace__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__sub__', '__subclasshook__', 'export', 'from_binary', 'get_affine', 'group', 'is_infinite', 'pt', 'pt_add', 'pt_add_inplace', 'pt_double', 'pt_double_inplace', 'pt_eq', 'pt_mul', 'pt_mul_inplace', 'pt_neg', 'pt_neg_inplace']


In [4]:
import syft as sy
from syft.lib import python

In [5]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_root_client()

# Q: Why is this needed?
#alice.root_verify_key = alice_client.verify_key  # inject 📡🔑 as 📍🗝
remote_python = alice_client.syft.lib.python

[2021-03-29T12:34:34.209752+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.0
[2021-03-29T12:34:34.210124+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.0
[2021-03-29T12:34:34.210391+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.0
[2021-03-29T12:34:34.210651+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.0
[2021-03-29T12:34:34.210885+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.0
[2021-03-29T12:34:34.211152+0530][CRITICAL][logger]][3309] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.0


In [6]:
sy.load('zksk')
sy.load('petlib')
zksk = alice_client.zksk
Secret = zksk.Secret
DLRep = zksk.DLRep

### Issue
> G, H = zksk.utils.make_generators(num=2, seed=42)

TypeError: cannot unpack non-iterable EcPtPointer object

Work around: 
> G = zksk.utils.make_generators(num=1, seed=42);
> H = zksk.utils.make_generators(num=1, seed=42)

In [7]:
# Setup: get the agreed group generators.
G, H = zksk.utils.make_generators(num=2, seed=42)
# Setup: define a randomizer with an unknown value.
r = Secret()

TypeError: cannot unpack non-iterable EcPtPointer object

In [9]:
G = zksk.utils.make_generators(num=1, seed=42)
H = zksk.utils.make_generators(num=1, seed=42)
dir(G)
type(G)

syft.proxy.petlib.ec.EcPtPointer

In [11]:
zksk = alice_client.zksk
Secret = zksk.Secret
DLRep = zksk.DLRep


### Issue 2:

`stmt = DLRep(C, r * H) | DLRep(C - G, r * H)`


> TypeError: 'Expression' object is not iterable
 

In [12]:
remote_python = alice_client.syft.lib.python
zksk = alice_client.zksk
Secret = zksk.Secret
DLRep = zksk.DLRep
# Setup: get the agreed group generators.
zksk = alice_client.zksk
Secret = zksk.Secret
DLRep = zksk.DLRep
# Setup: get the agreed group generators.
# G, H = zksk.utils.make_generators(num=2, seed=42)
# TypeError: cannot unpack non-iterable EcPtPointer object
G = zksk.utils.make_generators(num=1, seed=42)
H = zksk.utils.make_generators(num=1, seed=42)
# Setup: define a randomizer with an unknown value.
r = Secret()
# print(type(C),type(r), type(H), type(C - G))
# print(type( r * H))
print(type(H))
stmt = DLRep(C, r * H) | DLRep(C - G, r * H)
# Throws stack out error 

<class 'syft.proxy.petlib.ec.EcPtPointer'>


TypeError: 'Expression' object is not iterable

# Doubts:
    
1. Expectation in `LMW support`
2. Expectation in `Notebook Examples` 